### Exploratory Data Analysis of Soarin' Wait Times

#### Load modeling dataset from sqlite3

In [1]:
# Packages
import numpy as np
import pandas as pd
import sqlite3
import plotly.express as px

In [2]:
# Pull Soarnin' data
with open('../src/soarin_query.sql', 'r') as sqlfile:
    soarin_query = sqlfile.read()

with sqlite3.connect('../data/project_data.db') as conn:
    soarin_df = pd.read_sql(soarin_query, con=conn)

#### Training fold
The proposed modeling method will have a simplistic train/dev/test split in which training will be done on years 2015-2017. Data from 2018 will be used as a validation/dev set and 2019 will be used for testing. Given this method, exploratory analysis will be limited to 2015-2017 only in order to limit leakage of information from the dev/test sets into the training set.

In [3]:
# Subset data to only include what will become the training set
train_df = soarin_df[soarin_df.year_of_calendar.isin([2015,2016,2017])].reset_index(drop=True)

In [4]:
# View the first 5 rows
train_df.head()

,date_id,year_of_calendar,month_of_year,day_of_month,day_of_week,hour_of_day,minute_of_day,tmax,holiday_ind,unemp_local_change,unemp_natl_change,cpi_natl_change,avg_wait_time_prev,wait_time
0,2015-01-08,2015,1,8,3,8,56,62.0,0,0.2,0.2,0.598,30.625,10.0
1,2015-01-08,2015,1,8,3,9,0,62.0,0,0.2,0.2,0.598,57.333,10.0
2,2015-01-08,2015,1,8,3,9,7,62.0,0,0.2,0.2,0.598,57.333,10.0
3,2015-01-08,2015,1,8,3,9,14,62.0,0,0.2,0.2,0.598,57.333,10.0
4,2015-01-08,2015,1,8,3,9,21,62.0,0,0.2,0.2,0.598,57.333,15.0


#### Descriptive Statistics

In [5]:
# Describe df
train_df.describe()

,year_of_calendar,month_of_year,day_of_month,day_of_week,hour_of_day,minute_of_day,tmax,holiday_ind,unemp_local_change,unemp_natl_change,cpi_natl_change,avg_wait_time_prev,wait_time
count,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,124397.000000,120265.000000,124397.000000
mean,2015.964219,6.843557,15.790043,2.989630,14.445670,28.791048,85.391649,0.031432,-0.000775,-0.022719,0.105650,55.095278,55.338240
std,0.890531,3.316046,8.743438,1.993967,3.854057,17.978447,7.850860,0.174482,0.127656,0.173080,0.465975,27.028798,30.156481
min,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,52.000000,0.000000,-0.500000,-0.700000,-1.407000,5.000000,0.000000
25%,2015.000000,4.000000,8.000000,1.000000,11.000000,15.000000,82.000000,0.000000,0.000000,0.000000,0.000000,37.500000,35.000000
50%,2016.000000,7.000000,16.000000,3.000000,14.000000,28.000000,87.000000,0.000000,0.000000,0.000000,0.000000,52.000000,50.000000
75%,2017.000000,10.000000,23.000000,5.000000,18.000000,43.000000,91.000000,0.000000,0.000000,0.000000,0.000000,69.167000,70.000000
max,2017.000000,12.000000,31.000000,6.000000,23.000000,59.000000,100.000000,1.000000,0.300000,0.300000,1.339000,210.000000,240.000000


In [6]:
# Null percentage
train_df.apply(lambda x:pd.isnull(x).sum())/train_df.count()*100

date_id               0.000000
year_of_calendar      0.000000
month_of_year         0.000000
day_of_month          0.000000
day_of_week           0.000000
hour_of_day           0.000000
minute_of_day         0.000000
tmax                  0.000000
holiday_ind           0.000000
unemp_local_change    0.000000
unemp_natl_change     0.000000
cpi_natl_change       0.000000
avg_wait_time_prev    3.435746
wait_time             0.000000
dtype: float64

In [11]:
# Wait time distribution
fig = px.histogram(data_frame=train_df, x='wait_time', nbins=30, title='Wait Time Distribution')
fig.update_traces(marker_line_color='rgb(5,5,5)', marker_line_width=1)

In [10]:
# Quantiles
train_df.wait_time.quantile([i/10 for i in range(11)])

0.0      0.0
0.1     20.0
0.2     30.0
0.3     40.0
0.4     45.0
0.5     50.0
0.6     60.0
0.7     65.0
0.8     75.0
0.9     95.0
1.0    240.0
Name: wait_time, dtype: float64

In [23]:
# Month of year
context = train_df \
.groupby(['month_of_year'], as_index=False) \
.agg({'wait_time':'mean'})
fig = px.line(data_frame=context, x='month_of_year', y='wait_time', title='Average Wait Time by Month')
fig.update_traces(mode='markers+lines')